In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

In [2]:
def filter_res(df):
    df = df.loc[df['Comp'] == 'Premier League']
    df = df[['Date', 'Round', 'Result', 'Venue', 'GF', 'GA', 'xG', 'xGA']]
    return df

In [3]:
import os
import glob

path = 'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/match-by-match'
extension = 'csv'
os.chdir(path)
result = glob.glob('*.{}'.format(extension))
print(result)

['brighton-matches-s1.csv', 'brighton-matches-s2.csv', 'hudds-matches-s1.csv', 'hudds-matches-s2.csv', 'leeds-matches-2021.csv', 'leeds-matches-2122.csv', 'newcastle-matches-s1.csv', 'newcastle-matches-s2.csv', 'sheffutd-matches-s1.csv', 'sheffutd-matches-s2.csv', 'villa-matches-s1.csv', 'villa-matches-s2.csv', 'wolves-matches-s1.csv', 'wolves-matches-s2.csv']


In [4]:
lst = []
error_df = 'villa-matches-s1.csv'
for i in range(len(result)):
    if result[i] != error_df:
        df = pd.read_csv(result[i])
        val = filter_res(df)
        lst.append(val)
    else:
        df = pd.read_csv(result[i])
        df = df[['Date', 'Round', 'Result', 'Venue', 'GF', 'GA', 'xG', 'xGA']]
        lst.append(val)
        
print(len(lst))

14


In [5]:
lst[0]

,Date,Round,Result,Venue,GF,GA,xG,xGA
0,2017-08-12,Matchweek 1,L,Home,0,2,0.3,1.9
1,2017-08-19,Matchweek 2,L,Away,0,2,0.2,2.0
3,2017-08-26,Matchweek 3,D,Away,0,0,1.1,0.3
4,2017-09-09,Matchweek 4,W,Home,3,1,0.4,1.0
5,2017-09-15,Matchweek 5,L,Away,1,2,0.9,1.2
7,2017-09-24,Matchweek 6,W,Home,1,0,1.0,1.5
8,2017-10-01,Matchweek 7,L,Away,0,2,0.4,2.4
9,2017-10-15,Matchweek 8,D,Home,1,1,1.0,1.5
10,2017-10-20,Matchweek 9,W,Away,3,0,1.4,0.7
11,2017-10-29,Matchweek 10,D,Home,1,1,1.2,0.3


## Actual Points Calculator

Added to compare xP against actual to see how luck results were for a team.

In [6]:
def points_calculator(df):
    points = []
    result = list(df['Result'])
    for k in range(len(result)):
        if result[i] == "W":
            points.append(3)
        elif result[i] == "D":
            points.append(1)
        else:
            points.append(0)
    df['ActualPoints'] = points
    return df

### Expected Points Calculator
Typically an xP (expected points) calculator is based on each individual shot taken in a game by both teams. We don't have access to this data anywhere so instead we will follow another approach based on the overall team xG that utilizes a Poisson distribution seen [here](https://github.com/kostino/ExpectedPointsCalculator/blob/master/xPoints_Barcelona-RealMadrid_example.ipynb).

In [7]:
from scipy.stats import poisson
max_goals_pl = 9 # max goals scored in PL history is 9, assumed as no higher

def xp_calculator(df):
    points = []
    result = list(df['Result'])
    for k in range(len(result)):
        if result[k] == "W":
            points.append(3)
        elif result[k] == "D":
            points.append(1)
        else:
            points.append(0)

    xGLeeds = list(df['xG'])
    xGAway = list(df['xGA'])
    LeedsxP = []
    AwayxP = []
    for j in range(len(xGLeeds)):
        gProbsLeeds = [poisson.pmf(i, xGLeeds[j]) for i in range(max_goals_pl)]
        gProbsAway = [poisson.pmf(i, xGAway[j]) for i in range(max_goals_pl)]

        resultProbs = np.matmul(np.array(gProbsLeeds).reshape(max_goals_pl, 1),np.array(gProbsAway).reshape(1, max_goals_pl))

        LeedsWP = np.tril(resultProbs).sum() - np.trace(resultProbs)
        AwayWP = np.triu(resultProbs).sum() - np.trace(resultProbs)
        DrawP = np.trace(resultProbs)

        LeedsxPVals = LeedsWP * 3 + DrawP * 1
        AwayxPVals = AwayWP * 3 + DrawP * 1
        LeedsxP.append(LeedsxPVals)
        AwayxP.append(AwayxPVals)
    df['xPLeeds'] = LeedsxP
    df['xPOpp'] = AwayxP
    df['ActualPoints'] = points
    return df

In [8]:
outputdfs_xp = ['C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/brighton-matches-s1.csv', 
             'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/brighton-matches-s2.csv',
            'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/hudds-matches-s1.csv', 
             'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/hudds-matches-s2.csv',
             'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/leeds-matches-s1.csv', 
             'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/leeds-matches-s2.csv',
            'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/newcastle-matches-s1.csv', 
             'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/newcastle-matches-s2.csv',
            'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/sheffutd-matches-s1.csv', 
             'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/sheffutd-matches-s2.csv',
            'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/villa-matches-s1.csv', 
             'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/villa-matches-s2.csv',
            'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/wolves-matches-s1.csv', 
             'C:/Users/user/Desktop/xg-data-analysis/time-series-analysis/xPData/wolves-matches-s2.csv']

In [9]:
for i in range(len(lst)):
    xP_applied = xp_calculator(lst[i])
    xP_applied.to_csv(outputdfs_xp[i], index =  False)